## Our web-scraper is a (heavily) edited version of zameen-com-scrapper by [https://github.com/IIvexII/zameen-com-scrapper/blob/main/scraper.ipynb]. 

### We made modifications to the scrap() function to use BOTH HTML selectors, as well as JSON parsing and RegEx to extract far more data. Ours extracts the following data:
city	location	price	bedrooms	baths	size	title	page_url	date_added	property_type	latitude	longitude	number_of_photos	province	agency	purpose.

In total, these make **16 attributes**.
### Their version used only HTML selectors so it could just extract 6 attributes namely: 
(1)city, (2)location, (3)price, (4)number of bedrooms and (5)bathrooms, and (6)size of the property. 

### We kept the following functions from their script:
**convert_price()**: *converts "lakh/crore/arab" into integers*

**convert_size()**: *converts "kanal/marla" into same units (sqft)*

**text()**: *used to label datatypes*
 

In [2]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import concurrent.futures

In [3]:
def convert_price(price):
    """
    Convert crore, lakhs, millions and Thousand into numbers

    :param price: str
    :return: float
    """
    if price.endswith('Crore'):
        return round(float(price[:-5]) * 10000000)
    elif price.endswith('Lakh'):
        return round(float(price[:-4]) * 100000)
    elif price.endswith('Million'):
        return round(float(price[:-7]) * 1000000)
    elif price.endswith('Arab'):
        return round(float(price[:-4]) * 1000000000)
    elif price.endswith('Thousand'):
        return round(float(price[:-8]) * 1000)
    else:
        return round(float(price))

In [4]:
def convert_size(size):
    """
    Convert kanal merla into sqft

    :param size: str
    :return: float
    """
    if size.endswith('Marla'):
        return round(float(size[:-5].replace(",", "")) * 225)
    elif size.endswith('Kanal'):
        return round(float(size[:-5].replace(",", "")) * 4500)
    elif size.endswith('Sq. Yd.'):
        return round(float(size[:-7].replace(",", "")) * 9)
    else:
        return round(float(size))

In [5]:
def text(tag, datatype="str"):
    """
    This function will return the text of the tag.

    :param tag: tag object
    :param datatype: num or str or price, size
    :return: price in number or string
    """
    if tag is None and datatype == "num":
        return 0
    if datatype == "num":
        try:
            return int(tag.text.strip())
        except ValueError:
            return 0
    if tag is None and datatype == "str":
        return ""
    if datatype == "str":
        return tag.text.strip()
    if tag is None and datatype == "price":
        return 0.0
    if datatype == "price":
        return convert_price(tag.text.strip())
    if tag is None and datatype == "size":
        return 0.0
    if datatype == "size":
        return convert_size(tag.text.strip())

In [6]:
def scrap(city, pages_range):
    """
    This function will scrap the zameen.com website and
    return the list of houses information

    :param city: str
    :param pages_range: int
    :return: list
    """
    house_info = []

    for page_number in range(1, pages_range+1):
        url = f'https://www.zameen.com/Homes/{city}-{page_number}.html'
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        house_list = soup.select("main > div > div > div > div > ul > li")

        # store length of the previous length of house info list
        prev_len = len(house_info)

        for house in house_list:
            page_url = house.select_one("a[aria-label='Listing link']")
            if page_url:
                title_tag = house.select_one("h2[aria-label='Title']")
                page_url = page_url['href']
                page_url = f"https://www.zameen.com{page_url}"
                res2 = requests.get(page_url)
                abc = BeautifulSoup(res2.text, 'html.parser')
                script_content = abc.find('script').text
                
                latitude_pattern = r'"latitude":(\d+\.\d+)'
                latitude_match = re.search(latitude_pattern, script_content)
                if latitude_match:
                    latitude = latitude_match.group(1)
#                     print("Latitude:", latitude)
                else:
                    latitude = 0.0
    
                longitude_pattern = r'"longitude":(\d+\.\d+)'
                longitude_match = re.search(longitude_pattern, script_content)
                if longitude_match:
                    longitude = longitude_match.group(1)
#                     print("Longitude:", longitude)
                else:
                    longitude = 0.0
                    
                property_type_pattern = r'"property_type":"(.*?)"'
                property_type_match = re.search(property_type_pattern, script_content)
                if property_type_match:
                    property_type = property_type_match.group(1).replace(';', '')
#                     print("Property Type:", property_type)
                else:
                    propety_type = "undefined"
    
                number_of_photos_pattern = r'"number_of_photos":(\d+)'
                number_of_photos_match = re.search(number_of_photos_pattern, script_content)
                if number_of_photos_match:
                    number_of_photos = number_of_photos_match.group(1)
#                     print("Number of Photos:", number_of_photos)
                else:
                    number_of_photos = 0
                    
                loc_region_name_pattern = r'"loc_region_name":"(.*?)"'
                loc_region_name_match = re.search(loc_region_name_pattern, script_content)
                if loc_region_name_match:
                    loc_region_name = loc_region_name_match.group(1).replace(';', '')
#                     print("Location Region Name:", loc_region_name)
                else:
                    loc_region_name = ""
    
                marketed_by_pattern = r'"marketed_by":"(.*?)"'
                marketed_by_match = re.search(marketed_by_pattern, script_content)
                if marketed_by_match:
                    marketed_by = marketed_by_match.group(1)
                else:
                    marketed_by = ""
#                     print("Marketed By:", marketed_by)
                    
                purpose_pattern = r'"purpose":"(.*?)"'
                purpose_match = re.search(purpose_pattern, script_content)
                if purpose_match:
                    purpose = purpose_match.group(1)
#                     print("Purpose:", purpose)
                
                baths = house.select_one("span[aria-label='Baths']")
                beds = house.select_one("span[aria-label='Beds']")
                location = house.select_one("div[aria-label='Location']")
                price = house.select_one("span[aria-label='Price']")
                size = house.select_one("div[title]>div > div > span:nth-child(1)")
                date_added = house.select_one("span[aria-label='Listing creation date']")
            
            if price:
                if size is None:
                    size = location.parent.select_one(
                        "div:nth-child(2) > div > span:nth-child(3)")
                house_info.append(
                    {
                        "page_url": page_url,
                        "title": text(title_tag, datatype="str"),
                        "location": text(location),
                        "price": text(price, datatype="price"),
                        "bedrooms": text(beds, datatype="num"),
                        "baths": text(baths, datatype="num"),
                        "size": text(size, datatype="size"),
                        "date_added" : text(date_added, datatype="str"),
                        "purpose" : purpose,
                        "property_type" : property_type,
                        "latitude" : latitude,
                        "longitude" : longitude,
                        "number_of_photos" : number_of_photos,
                        "province" : loc_region_name,
                        "agency" : marketed_by
                        
                    }
                )

        # get out of the loop if the last accessed page
        # does not exist to avoid useless requests because
        # next pages will not exist as well
        if len(house_info) == prev_len:
            break

    return house_info


# At this point we could not figure out how to implement this code asynchronously. 
So we did the next best thing which was to make a separate cell for each city and call them simultaneously. 
This means we will end up with a separate csv file for each city, which we can join/merge during the EDA process. 
#### This version of the script only scrapes 10 pages per city, this was due to our time constraints, otherwise this could easily be scaled up to scraping as many pages as needed. 

Time to press (Shift+Enter)*23

:p

# 1- Karachi

In [6]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 2, 'name': 'Karachi'},
#         {'id': 3, 'name': 'Islamabad'},
#         {'id': 1, 'name': 'Lahore'},
#         {'id': 15, 'name': 'Multan'},
#         {'id': 16, 'name': 'Faisalabad'},
#         {'id': 17, 'name': 'Peshawar'},
#         {'id': 18, 'name': 'Quetta'},
#         {'id': 19, 'name': 'Jhelum'},
#         {'id': 20, 'name': 'Gujrat'},
#         {'id': 23, 'name': 'Bahawalpur'},
#         {'id': 26, 'name': 'Dera_Gazi_Khan'},
#         {'id': 30, 'name': 'Hyderabad'},
#         {'id': 41, 'name': 'Rawalpindi'},
#         {'id': 44, 'name': 'Sheikhupura'},
#         {'id': 36, 'name': 'Murree'},
#         {'id': 327, 'name': 'Gujranwala'},
#         {'id': 385, 'name': 'Abbottabad'},
#         {'id': 459, 'name': 'Wah'},
#         {'id': 470, 'name': 'Okara'},
#         {'id': 480, 'name': 'Sialkot'},
#         {'id': 778, 'name': 'Sargodha'},
#         {'id': 782, 'name': 'Sahiwal'},
#         {'id': 1233, 'name': 'Attock'},
    ]

        
    with open("Karachi.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Karachi-2-1.html
https://www.zameen.com/Homes/Karachi-2-2.html
https://www.zameen.com/Homes/Karachi-2-3.html
https://www.zameen.com/Homes/Karachi-2-4.html
https://www.zameen.com/Homes/Karachi-2-5.html
https://www.zameen.com/Homes/Karachi-2-6.html
https://www.zameen.com/Homes/Karachi-2-7.html
https://www.zameen.com/Homes/Karachi-2-8.html
https://www.zameen.com/Homes/Karachi-2-9.html
https://www.zameen.com/Homes/Karachi-2-10.html
**Done!**


# 2- Islamabad

In [7]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 3, 'name': 'Islamabad'},
    ]

        
    with open("Islamabad.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Islamabad-3-1.html
https://www.zameen.com/Homes/Islamabad-3-2.html
https://www.zameen.com/Homes/Islamabad-3-3.html
https://www.zameen.com/Homes/Islamabad-3-4.html
https://www.zameen.com/Homes/Islamabad-3-5.html
https://www.zameen.com/Homes/Islamabad-3-6.html
https://www.zameen.com/Homes/Islamabad-3-7.html
https://www.zameen.com/Homes/Islamabad-3-8.html
https://www.zameen.com/Homes/Islamabad-3-9.html
https://www.zameen.com/Homes/Islamabad-3-10.html
**Done!**


# 3- Lahore

In [8]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 1, 'name': 'Lahore'},
    ]

        
    with open("Lahore.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Lahore-1-1.html
https://www.zameen.com/Homes/Lahore-1-2.html
https://www.zameen.com/Homes/Lahore-1-3.html
https://www.zameen.com/Homes/Lahore-1-4.html
https://www.zameen.com/Homes/Lahore-1-5.html
https://www.zameen.com/Homes/Lahore-1-6.html
https://www.zameen.com/Homes/Lahore-1-7.html
https://www.zameen.com/Homes/Lahore-1-8.html
https://www.zameen.com/Homes/Lahore-1-9.html
https://www.zameen.com/Homes/Lahore-1-10.html
**Done!**


# 4- Multan

In [9]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 15, 'name': 'Multan'},
    ]

        
    with open("Multan.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Multan-15-1.html
https://www.zameen.com/Homes/Multan-15-2.html
https://www.zameen.com/Homes/Multan-15-3.html
https://www.zameen.com/Homes/Multan-15-4.html
https://www.zameen.com/Homes/Multan-15-5.html
https://www.zameen.com/Homes/Multan-15-6.html
https://www.zameen.com/Homes/Multan-15-7.html
https://www.zameen.com/Homes/Multan-15-8.html
https://www.zameen.com/Homes/Multan-15-9.html
https://www.zameen.com/Homes/Multan-15-10.html
**Done!**


# 5- Faisalabad

In [10]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 16, 'name': 'Faisalabad'},
    ]

        
    with open("Faisalabad.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Faisalabad-16-1.html
https://www.zameen.com/Homes/Faisalabad-16-2.html
https://www.zameen.com/Homes/Faisalabad-16-3.html
https://www.zameen.com/Homes/Faisalabad-16-4.html
https://www.zameen.com/Homes/Faisalabad-16-5.html
https://www.zameen.com/Homes/Faisalabad-16-6.html
https://www.zameen.com/Homes/Faisalabad-16-7.html
https://www.zameen.com/Homes/Faisalabad-16-8.html
https://www.zameen.com/Homes/Faisalabad-16-9.html
https://www.zameen.com/Homes/Faisalabad-16-10.html
**Done!**


# 6- Peshawar

In [11]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 17, 'name': 'Peshawar'},
    ]

        
    with open("Peshawar.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Peshawar-17-1.html
https://www.zameen.com/Homes/Peshawar-17-2.html
https://www.zameen.com/Homes/Peshawar-17-3.html
https://www.zameen.com/Homes/Peshawar-17-4.html
https://www.zameen.com/Homes/Peshawar-17-5.html
https://www.zameen.com/Homes/Peshawar-17-6.html
https://www.zameen.com/Homes/Peshawar-17-7.html
https://www.zameen.com/Homes/Peshawar-17-8.html
https://www.zameen.com/Homes/Peshawar-17-9.html
https://www.zameen.com/Homes/Peshawar-17-10.html
**Done!**


# 7- Quetta

In [12]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 18, 'name': 'Quetta'},
    ]

        
    with open("Quetta.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Quetta-18-1.html
https://www.zameen.com/Homes/Quetta-18-2.html
https://www.zameen.com/Homes/Quetta-18-3.html
https://www.zameen.com/Homes/Quetta-18-4.html
https://www.zameen.com/Homes/Quetta-18-5.html
https://www.zameen.com/Homes/Quetta-18-6.html
https://www.zameen.com/Homes/Quetta-18-7.html
https://www.zameen.com/Homes/Quetta-18-8.html
https://www.zameen.com/Homes/Quetta-18-9.html
https://www.zameen.com/Homes/Quetta-18-10.html
**Done!**


# 8- Jhelum

In [13]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 19, 'name': 'Jhelum'},
    ]

        
    with open("Jhelum.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Jhelum-19-1.html
https://www.zameen.com/Homes/Jhelum-19-2.html
https://www.zameen.com/Homes/Jhelum-19-3.html
https://www.zameen.com/Homes/Jhelum-19-4.html
https://www.zameen.com/Homes/Jhelum-19-5.html
https://www.zameen.com/Homes/Jhelum-19-6.html
https://www.zameen.com/Homes/Jhelum-19-7.html
https://www.zameen.com/Homes/Jhelum-19-8.html
https://www.zameen.com/Homes/Jhelum-19-9.html
https://www.zameen.com/Homes/Jhelum-19-10.html
**Done!**


# 9- Gujrat

In [14]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 20, 'name': 'Gujrat'},
    ]

        
    with open("Gujrat.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Gujrat-20-1.html
https://www.zameen.com/Homes/Gujrat-20-2.html
https://www.zameen.com/Homes/Gujrat-20-3.html
https://www.zameen.com/Homes/Gujrat-20-4.html
https://www.zameen.com/Homes/Gujrat-20-5.html
https://www.zameen.com/Homes/Gujrat-20-6.html
https://www.zameen.com/Homes/Gujrat-20-7.html
https://www.zameen.com/Homes/Gujrat-20-8.html
https://www.zameen.com/Homes/Gujrat-20-9.html
https://www.zameen.com/Homes/Gujrat-20-10.html
**Done!**


# 10- Bahawalpur

In [15]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 23, 'name': 'Bahawalpur'},
    ]

        
    with open("Bahawalpur.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Bahawalpur-23-1.html
https://www.zameen.com/Homes/Bahawalpur-23-2.html
https://www.zameen.com/Homes/Bahawalpur-23-3.html
https://www.zameen.com/Homes/Bahawalpur-23-4.html
https://www.zameen.com/Homes/Bahawalpur-23-5.html
https://www.zameen.com/Homes/Bahawalpur-23-6.html
https://www.zameen.com/Homes/Bahawalpur-23-7.html
https://www.zameen.com/Homes/Bahawalpur-23-8.html
https://www.zameen.com/Homes/Bahawalpur-23-9.html
https://www.zameen.com/Homes/Bahawalpur-23-10.html
**Done!**


# 11- DGK

In [16]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 26, 'name': 'Dera_Gazi_Khan'},
    ]

        
    with open("Dera_Gazi_Khan.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Dera_Gazi_Khan-26-1.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-2.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-3.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-4.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-5.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-6.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-7.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-8.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-9.html
https://www.zameen.com/Homes/Dera_Gazi_Khan-26-10.html
**Done!**


# 12- Hyderabad

In [17]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 30, 'name': 'Hyderabad'},
    ]

        
    with open("Hyderabad.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Hyderabad-30-1.html
https://www.zameen.com/Homes/Hyderabad-30-2.html
https://www.zameen.com/Homes/Hyderabad-30-3.html
https://www.zameen.com/Homes/Hyderabad-30-4.html
https://www.zameen.com/Homes/Hyderabad-30-5.html
https://www.zameen.com/Homes/Hyderabad-30-6.html
https://www.zameen.com/Homes/Hyderabad-30-7.html
https://www.zameen.com/Homes/Hyderabad-30-8.html
https://www.zameen.com/Homes/Hyderabad-30-9.html
https://www.zameen.com/Homes/Hyderabad-30-10.html
**Done!**


# 13- Rawalpindi

In [18]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 41, 'name': 'Rawalpindi'},
    ]

        
    with open("Rawalpindi.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Rawalpindi-41-1.html
https://www.zameen.com/Homes/Rawalpindi-41-2.html
https://www.zameen.com/Homes/Rawalpindi-41-3.html
https://www.zameen.com/Homes/Rawalpindi-41-4.html
https://www.zameen.com/Homes/Rawalpindi-41-5.html
https://www.zameen.com/Homes/Rawalpindi-41-6.html
https://www.zameen.com/Homes/Rawalpindi-41-7.html
https://www.zameen.com/Homes/Rawalpindi-41-8.html
https://www.zameen.com/Homes/Rawalpindi-41-9.html
https://www.zameen.com/Homes/Rawalpindi-41-10.html
**Done!**


# 14- Sheikhupura

In [19]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 44, 'name': 'Sheikhupura'},
    ]

        
    with open("Sheikhupura.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Sheikhupura-44-1.html
https://www.zameen.com/Homes/Sheikhupura-44-2.html
https://www.zameen.com/Homes/Sheikhupura-44-3.html
https://www.zameen.com/Homes/Sheikhupura-44-4.html
https://www.zameen.com/Homes/Sheikhupura-44-5.html
https://www.zameen.com/Homes/Sheikhupura-44-6.html
https://www.zameen.com/Homes/Sheikhupura-44-7.html
https://www.zameen.com/Homes/Sheikhupura-44-8.html
https://www.zameen.com/Homes/Sheikhupura-44-9.html
https://www.zameen.com/Homes/Sheikhupura-44-10.html
**Done!**


# 15- Murree

In [20]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 36, 'name': 'Murree'},
    ]

        
    with open("Murree.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Murree-36-1.html
https://www.zameen.com/Homes/Murree-36-2.html
https://www.zameen.com/Homes/Murree-36-3.html
https://www.zameen.com/Homes/Murree-36-4.html
https://www.zameen.com/Homes/Murree-36-5.html
https://www.zameen.com/Homes/Murree-36-6.html
https://www.zameen.com/Homes/Murree-36-7.html
https://www.zameen.com/Homes/Murree-36-8.html
https://www.zameen.com/Homes/Murree-36-9.html
https://www.zameen.com/Homes/Murree-36-10.html
**Done!**


# 16- Gujranwala

In [21]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 327, 'name': 'Gujranwala'},
    ]

        
    with open("Gujranwala.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Gujranwala-327-1.html
https://www.zameen.com/Homes/Gujranwala-327-2.html
https://www.zameen.com/Homes/Gujranwala-327-3.html
https://www.zameen.com/Homes/Gujranwala-327-4.html
https://www.zameen.com/Homes/Gujranwala-327-5.html
https://www.zameen.com/Homes/Gujranwala-327-6.html
https://www.zameen.com/Homes/Gujranwala-327-7.html
https://www.zameen.com/Homes/Gujranwala-327-8.html
https://www.zameen.com/Homes/Gujranwala-327-9.html
https://www.zameen.com/Homes/Gujranwala-327-10.html
**Done!**


# 17- Abbottabad

In [22]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 385, 'name': 'Abbottabad'},
    ]

        
    with open("Abbottabad.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Abbottabad-385-1.html
https://www.zameen.com/Homes/Abbottabad-385-2.html
https://www.zameen.com/Homes/Abbottabad-385-3.html
https://www.zameen.com/Homes/Abbottabad-385-4.html
https://www.zameen.com/Homes/Abbottabad-385-5.html
https://www.zameen.com/Homes/Abbottabad-385-6.html
https://www.zameen.com/Homes/Abbottabad-385-7.html
https://www.zameen.com/Homes/Abbottabad-385-8.html
https://www.zameen.com/Homes/Abbottabad-385-9.html
https://www.zameen.com/Homes/Abbottabad-385-10.html
**Done!**


# 18- Wah

In [23]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 459, 'name': 'Wah'},
    ]

        
    with open("Wah.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Wah-459-1.html
https://www.zameen.com/Homes/Wah-459-2.html
https://www.zameen.com/Homes/Wah-459-3.html
https://www.zameen.com/Homes/Wah-459-4.html
https://www.zameen.com/Homes/Wah-459-5.html
https://www.zameen.com/Homes/Wah-459-6.html
https://www.zameen.com/Homes/Wah-459-7.html
https://www.zameen.com/Homes/Wah-459-8.html
https://www.zameen.com/Homes/Wah-459-9.html
https://www.zameen.com/Homes/Wah-459-10.html
**Done!**


# 19- Okara

In [24]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 470, 'name': 'Okara'},
    ]

        
    with open("Okara.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Okara-470-1.html
https://www.zameen.com/Homes/Okara-470-2.html
https://www.zameen.com/Homes/Okara-470-3.html
https://www.zameen.com/Homes/Okara-470-4.html
https://www.zameen.com/Homes/Okara-470-5.html
https://www.zameen.com/Homes/Okara-470-6.html
https://www.zameen.com/Homes/Okara-470-7.html
https://www.zameen.com/Homes/Okara-470-8.html
https://www.zameen.com/Homes/Okara-470-9.html
https://www.zameen.com/Homes/Okara-470-10.html
**Done!**


# 20- Sialkot

In [25]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 480, 'name': 'Sialkot'},
    ]

        
    with open("Sialkot.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Sialkot-480-1.html
https://www.zameen.com/Homes/Sialkot-480-2.html
https://www.zameen.com/Homes/Sialkot-480-3.html
https://www.zameen.com/Homes/Sialkot-480-4.html
https://www.zameen.com/Homes/Sialkot-480-5.html
https://www.zameen.com/Homes/Sialkot-480-6.html
https://www.zameen.com/Homes/Sialkot-480-7.html
https://www.zameen.com/Homes/Sialkot-480-8.html
https://www.zameen.com/Homes/Sialkot-480-9.html
https://www.zameen.com/Homes/Sialkot-480-10.html
**Done!**


# 21- Sargodha

In [26]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 778, 'name': 'Sargodha'},
    ]

        
    with open("Sargodha.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Sargodha-778-1.html
https://www.zameen.com/Homes/Sargodha-778-2.html
https://www.zameen.com/Homes/Sargodha-778-3.html
https://www.zameen.com/Homes/Sargodha-778-4.html
https://www.zameen.com/Homes/Sargodha-778-5.html
https://www.zameen.com/Homes/Sargodha-778-6.html
https://www.zameen.com/Homes/Sargodha-778-7.html
https://www.zameen.com/Homes/Sargodha-778-8.html
https://www.zameen.com/Homes/Sargodha-778-9.html
https://www.zameen.com/Homes/Sargodha-778-10.html
**Done!**


# 22- Sahiwal

In [27]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 782, 'name': 'Sahiwal'},
    ]

        
    with open("Sahiwal.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Sahiwal-782-1.html
https://www.zameen.com/Homes/Sahiwal-782-2.html
https://www.zameen.com/Homes/Sahiwal-782-3.html
https://www.zameen.com/Homes/Sahiwal-782-4.html
https://www.zameen.com/Homes/Sahiwal-782-5.html
https://www.zameen.com/Homes/Sahiwal-782-6.html
https://www.zameen.com/Homes/Sahiwal-782-7.html
https://www.zameen.com/Homes/Sahiwal-782-8.html
https://www.zameen.com/Homes/Sahiwal-782-9.html
https://www.zameen.com/Homes/Sahiwal-782-10.html
**Done!**


# 23- Attock

In [28]:
if __name__ == "__main__":
    house_info = []

    cities = [
        {'id': 1233, 'name': 'Attock'},
    ]

        
    with open("Attock.csv", "w", newline='') as f:
        csv_writer = csv.writer(f)

        # Write CSV header
        csv_writer.writerow(["city", "location", "price", "bedrooms", "baths", "size", "title", "page_url", "date_added", "property_type", "latitude", "longitude", "number_of_photos", "province", "agency", "purpose"])

        for city in cities:
            city_name = city.get('name')
            city_info = scrap(f"{city_name}-{city.get('id')}", 10)
            for info in city_info:
                csv_writer.writerow([city_name, info.get('location'), info.get('price'), info.get('bedrooms'), info.get('baths'), info.get('size'), 
                                      info.get('title'), info.get('page_url'), info.get('date_added'), info.get('property_type'), info.get('latitude'), info.get('longitude'), info.get('number_of_photos'), info.get('province'), info.get('agency'), info.get('purpose')])
        
        
print("**Done!**")

https://www.zameen.com/Homes/Attock-1233-1.html
https://www.zameen.com/Homes/Attock-1233-2.html
https://www.zameen.com/Homes/Attock-1233-3.html
https://www.zameen.com/Homes/Attock-1233-4.html
https://www.zameen.com/Homes/Attock-1233-5.html
https://www.zameen.com/Homes/Attock-1233-6.html
https://www.zameen.com/Homes/Attock-1233-7.html
https://www.zameen.com/Homes/Attock-1233-8.html
https://www.zameen.com/Homes/Attock-1233-9.html
https://www.zameen.com/Homes/Attock-1233-10.html
**Done!**


# The end